In [41]:
import geopandas as gpd
import folium

# Load your exported Overpass Turbo data
gdf = gpd.read_file("export.geojson")

# Optional: inspect what columns you have
gdf.head()


,id,@id,abandoned:historic,access,addr:city,addr:city:en,addr:country,addr:floor,addr:housenumber,addr:place,...,tourism,type,website,wheelchair,wikidata,wikimedia_commons,wikipedia,wikipedia:ru,@geometry,geometry
0,relation/2312684,relation/2312684,None,None,None,None,None,None,None,None,...,None,multipolygon,None,None,Q59590271,None,None,None,center,POINT (69.24304 41.32475)
1,relation/3412518,relation/3412518,None,None,None,None,None,None,101,None,...,museum,multipolygon,None,None,None,None,None,None,center,POINT (69.28048 41.30536)
2,way/31962395,way/31962395,None,None,Toshkent,Tashkent,None,None,28,None,...,None,None,https://gabt.uz/,None,Q3354568,Category:Alisher Navoi Opera and Ballet Theatre,ru:Большой театр имени Алишера Навои,None,center,POINT (69.27152 41.30922)
3,way/31972619,way/31972619,None,None,Toshkent,Tashkent,None,None,1,None,...,museum,None,None,None,Q4120971,Category:Amir Timur Museum,uz:Temuriylar tarixi davlat muzeyi,None,center,POINT (69.27899 41.31368)
4,way/31974956,way/31974956,None,None,Toshkent,None,None,None,3,None,...,museum,None,https://www.history-museum.uz/,None,Q1835285,Category:State Museum of History of Uzbekistan,en:State Museum of History of Uzbekistan,None,center,POINT (69.26933 41.31162)


In [42]:
art_gdf = gdf[
    gdf.apply(
        lambda row: any(
            str(row.get(c, "")).lower() in [
                "museum", "artwork", "gallery", "monument", "memorial"
            ]
            for c in ["tourism", "amenity", "artwork_type"]
        ),
        axis=1
    )
]


In [43]:
# Center map roughly around Tashkent
m = folium.Map(location=[41.3111, 69.2797], zoom_start=12, tiles="cartodb positron")

# Add points
for _, row in art_gdf.iterrows():
    lat, lon = row.geometry.y, row.geometry.x
    name = row.get("name", "Unnamed site")
    cat = row.get("tourism") or row.get("amenity") or row.get("artwork_type")
    desc = "<br>".join([f"<b>{k}</b>: {v}" for k, v in row.items() if isinstance(v, str) and v])

    folium.CircleMarker(
        location=[lat, lon],
        radius=6,
        color="crimson",
        fill=True,
        fill_opacity=0.8,
        popup=folium.Popup(f"<b>{name}</b><br>{cat}<br><small>{desc}</small>", max_width=250),
    ).add_to(m)

m


In [44]:
import random

# Assign random colors by main category
cats = art_gdf['tourism'].fillna(art_gdf['amenity']).fillna(art_gdf['artwork_type'])
unique_cats = cats.unique()
colors = {cat: f"#{random.randint(0, 0xFFFFFF):06x}" for cat in unique_cats}

m = folium.Map(location=[41.3111, 69.2797], zoom_start=12, tiles="cartodb dark_matter")

for _, row in art_gdf.iterrows():
    cat = row.get("tourism") or row.get("amenity") or row.get("artwork_type") or "other"
    color = colors.get(cat, "#ffcc00")
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=6,
        color=color,
        fill=True,
        fill_opacity=0.8,
        popup=row.get("name", "Unnamed site"),
    ).add_to(m)

m


In [45]:
category_styles = {
    "museum": {"icon": "university", "color": "darkblue"},
    "gallery": {"icon": "palette", "color": "purple"},
    "artwork": {"icon": "paint-brush", "color": "orange"},
    "monument": {"icon": "landmark", "color": "darkred"},
    "memorial": {"icon": "cross", "color": "gray"},
}


In [46]:
for _, row in art_gdf.iterrows():
    lat, lon = row.geometry.y, row.geometry.x
    cat = (row.get("tourism") or row.get("amenity") or row.get("artwork_type") or "other").lower()
    name = row.get("name", "Unnamed site")
    desc = "<br>".join(
        [f"<b>{k}</b>: {v}" for k, v in row.items() if isinstance(v, str) and v]
    )

    style = category_styles.get(cat, {"icon": "circle", "color": "lightgray"})

    folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(color=style["color"], icon=style["icon"], prefix="fa"),
        popup=folium.Popup(f"<b>{name}</b><br>{cat}<br><small>{desc}</small>", max_width=300),
    ).add_to(m)


In [48]:
legend_html = """
<div style="
position: fixed; 
bottom: 40px; left: 40px; width: 180px; 
background-color: rgba(255,255,255,0.8); 
padding: 10px; 
border-radius: 10px;
box-shadow: 0 0 10px rgba(0,0,0,0.3);
z-index:9999;
font-size:14px;">
<b>Legend</b><br>
<i class="fa fa-university" style="color:darkblue"></i> Museum<br>
<i class="fa fa-palette" style="color:purple"></i> Gallery<br>
<i class="fa fa-paint-brush" style="color:orange"></i> Artwork<br>
<i class="fa fa-landmark" style="color:darkred"></i> Monument<br>
<i class="fa fa-cross" style="color:gray"></i> Memorial<br>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))


In [49]:
m

In [50]:
m.save("Day 1 - art points")

In [51]:
import geopandas as gpd

# Load the shapefile (admin boundaries)
boundaries = gpd.read_file("uzb_admbnda_adm1_2018b.shp")


In [52]:
boundaries

,ADM0_EN,ADM0_RU,ADM0_UZ,ADM0ALT1EN,ADM0_PCODE,ADM1_EN,ADM1_RU,ADM1_UZ,ADM1TYPE_E,ADM1TYPE_R,ADM1TYPE_U,ADM1_PCODE,geometry
0,Uzbekistan,Республика Узбекистан,O'zbekiston Respublikasi,Republic of Uzbekistan,UZ,Andizhan,Андижанская,Andijon,province,область,viloyati,UZ03,"POLYGON ((72.35866 41.06426, 72.36182 41.06137..."
1,Uzbekistan,Республика Узбекистан,O'zbekiston Respublikasi,Republic of Uzbekistan,UZ,Bukhara,Бухарская,Buxoro,province,область,viloyati,UZ06,"POLYGON ((63.6738 40.67516, 63.68308 40.65618,..."
2,Uzbekistan,Республика Узбекистан,O'zbekiston Respublikasi,Republic of Uzbekistan,UZ,Dzhizak,Джизакская,Jizzax,province,область,viloyati,UZ08,"POLYGON ((67.32946 41.20266, 67.34834 41.20059..."
3,Uzbekistan,Республика Узбекистан,O'zbekiston Respublikasi,Republic of Uzbekistan,UZ,Fergana,Ферганская,Farg'ona,province,область,viloyati,UZ30,"MULTIPOLYGON (((71.83504 39.98598, 71.8387 39...."
4,Uzbekistan,Республика Узбекистан,O'zbekiston Respublikasi,Republic of Uzbekistan,UZ,Kashkadarya,Кашкадарьинская,Qashqadaryo,province,область,viloyati,UZ10,"POLYGON ((65.75394 39.51051, 65.76572 39.49956..."
5,Uzbekistan,Республика Узбекистан,O'zbekiston Respublikasi,Republic of Uzbekistan,UZ,Khorezm,Хорезмская,Xorazm,province,область,viloyati,UZ33,"POLYGON ((60.25193 41.95384, 60.27243 41.9343,..."
6,Uzbekistan,Республика Узбекистан,O'zbekiston Respublikasi,Republic of Uzbekistan,UZ,Namangan,Наманганская,Namangan,province,область,viloyati,UZ14,"POLYGON ((71.70757 41.42406, 71.71663 41.41976..."
7,Uzbekistan,Республика Узбекистан,O'zbekiston Respublikasi,Republic of Uzbekistan,UZ,Navoi,Навоийская,Navoiy,province,область,viloyati,UZ12,"POLYGON ((65.25539 43.60787, 65.26723 43.5886,..."
8,Uzbekistan,Республика Узбекистан,O'zbekiston Respublikasi,Republic of Uzbekistan,UZ,Republic of Karakalpakstan,Республика Каракалпакстан,Qoraqalpog'iston Respublikasi,autonomous republic,автономная республика,muxtor respublikasi,UZ35,"POLYGON ((61.04586 44.39032, 61.04997 44.3863,..."
9,Uzbekistan,Республика Узбекистан,O'zbekiston Respublikasi,Republic of Uzbekistan,UZ,Samarkand,Самаркандская,Samarqand,province,область,viloyati,UZ18,"POLYGON ((66.43228 40.59085, 66.45001 40.58405..."


In [55]:
import folium

# --- add boundary layer ---
folium.GeoJson(
    tashkent_boundary if 'tashkent_boundary' in locals() else boundaries,
    name="Administrative Boundary",
    style_function=lambda x: {
        'fillColor': 'none',
        'color': 'white',
        'weight': 2,
        'opacity': 0.8
    }
).add_to(m)


In [56]:
m